In [1]:
import Diffusion_1D_conditioned
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from conditionedDataset import AirfoilDataset

def plot_airfoils(airfoil_x, airfoil_y):
    '''
    plot airfoils: no need to modify 
    '''
    idx = 0
    fig, ax = plt.subplots(nrows=32, ncols=4, figsize=(15, 80))
    for row in ax:
        for col in row:
            col.scatter(airfoil_x, airfoil_y[idx, :], s=0.6, c='black')
            col.axis('off')
            col.axis('equal')
            idx += 1
    plt.show()

In [3]:
dataset = AirfoilDataset(path='cleaned_airfoils/')
airfoil_x = dataset.get_x()
y_coordinates = dataset.get_y()
airfoil_dim = airfoil_x.shape[0]
print("airfoil_dim: ",airfoil_dim)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)



airfoil_dim:  200


In [4]:
model = Diffusion_1D_conditioned.Unet1D(
    dim=8,  # Adjust based on the complexity of your data
    channels=1,  # Single channel for y-coordinates
    dim_mults=(1, 2, 4, 8),  # Example configuration; adjust as necessary
    resnet_block_groups=1
)



/home/reid/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [5]:
diffusion_model = Diffusion_1D_conditioned.GaussianDiffusion1D(
    model,
    timesteps=1000, 
    seq_length=200,
    objective='pred_noise'
)

In [6]:
trainer = Diffusion_1D_conditioned.Trainer1D(
    diffusion_model=diffusion_model,
    dataset=dataset,
    train_batch_size=64,
    train_lr=1e-5,
    train_num_steps=100000,
)
trainer.train()

/home/reid/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(


  0%|          | 0/100000 [00:00<?, ?it/s]

CL/CD values at index 57: tensor([0.0349, 0.0264])CL/CD values at index 443: tensor([-0.1352,  0.0651])CL/CD values at index 639: tensor([-0.0136,  0.0389])CL/CD values at index 704: tensor([-0.1297,  0.0513])CL/CD values at index 808: tensor([-0.1143,  0.0418])CL/CD values at index 761: tensor([-0.0804,  0.0424])CL/CD values at index 498: tensor([-0.1431,  0.0483])CL/CD values at index 92: tensor([-0.1544,  0.0440])CL/CD values at index 95: tensor([-0.0302,  0.0513])CL/CD values at index 16: tensor([0.1466, 0.0260])







CL/CD values at index 649: tensor([0.0304, 0.0569])CL/CD values at index 355: tensor([0.0286, 0.0301])CL/CD values at index 1193: tensor([0.0801, 0.0527])
CL/CD values at index 699: tensor([-0.0361,  0.0260])CL/CD values at index 554: tensor([-0.0440,  0.1084])
CL/CD values at index 450: tensor([-0.3597,  0.0706])CL/CD values at index 911: tensor([-0.0096,  0.0262])CL/CD values at index 67: tensor([0.1735, 0.0474])CL/CD values at index 962: tensor([0.1005, 0.0471])


AssertionError: Cl/Cd must be provided

In [ ]:
sampled_airfoils = diffusion_model.sample(128)
sampled_airfoils = sampled_airfoils.to('cpu')
plot_airfoils(airfoil_x, sampled_airfoils.squeeze().detach().numpy())


# To generate airfoils from an already trained model

In [ ]:
import torch
import Diffusion_1D
import numpy as np
import matplotlib.pyplot as plt

# Assuming the model class is defined somewhere in your script as `Unet1D`
# and the diffusion model class is defined as `GaussianDiffusion1D`

# Initialize your model architecture
model = Diffusion_1D.Unet1D(
    dim=8,
    channels=1,
    dim_mults=(1, 2, 4, 8),
    resnet_block_groups=1
)


# Initialize the diffusion model
diffusion_model = Diffusion_1D.GaussianDiffusion1D(
    model,
    timesteps=1000, 
    seq_length=200,
    objective='pred_noise'
)

# Load the trained parameters into your model
checkpoint = torch.load("/home/reid/Airfoil_Diffusion_Generator/results/model-10.pt")
print(checkpoint.keys())
# Assuming your model is an instance of Unet1D and checkpoint is loaded as shown

if 'model' in checkpoint:
    state_dict = {key.replace('model.', ''): value for key, value in checkpoint['model'].items()}
    model.load_state_dict(state_dict, strict=False)  # Set strict to False
else:
    model.load_state_dict(checkpoint, strict=False)  # Set strict to False

model.eval()

# If the model was trained with specific configurations or wrapped within another module, adjust accordingly

# Now the model is ready to generate data
# For example:
sampled_airfoils = diffusion_model.sample(128)


dict_keys(['step', 'model', 'opt', 'ema', 'scaler', 'version'])


sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
dataset = AirfoilDataset(path='cleaned_airfoils/')
airfoil_x = dataset.get_x()
plot_airfoils(airfoil_x, sampled_airfoils.squeeze().detach().numpy())

In [38]:
sampled_airfoils = sampled_airfoils.to('cpu')
airfoil_values = sampled_airfoils.squeeze().detach().numpy()
np.savetxt("sampled_airfoils.csv", airfoil_values, delimiter=",")

In [ ]:
import csv

# Load the dataset to get shared x values
dataset = AirfoilDataset(path='cleaned_airfoils/')
airfoil_x = dataset.get_x()

# save_dir = "gen_airfoils/"
save_dir = "gen_airfoils/"

with open("sampled_airfoils.csv", "r") as csvfile:
    csvreader = csv.reader(csvfile)
    gen_num = 0
    for row in csvreader:
        y_coordinates = []
        x_coordinates = airfoil_x.tolist()
        for val in row:
            y_coordinates.append(float(val))
        y_shift = y_coordinates[99]
        y_coordinates = [y - y_shift for y in y_coordinates]

        with open(save_dir + f"gen_airfoil_{gen_num}.dat", "w") as f:
            for x,y in zip(x_coordinates, y_coordinates):
                print("writing: ",x,y, "to file", save_dir + f"gen_airfoil_{gen_num}.dat")
                f.write(f"{x:.8f} {y:.8f}\n")
        gen_num += 1



